<a href="https://colab.research.google.com/github/utensil/llm-playground/blob/main/notebooks/axolotl/runpod/axolotl-falcon-40b-qlora-deepspeed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetuning falcon-40b

- Axolotl+QLoRA
- minotaur datasets
- deepspeed ZeRO 3 8xGPU

<!-- https://jupyterlab.readthedocs.io/en/stable/user/commands.html#commands-list -->
<button data-commandLinker-command="apputils:change-theme" data-commandlinker-args='{"theme": "JupyterLab Dark"}' href="#">Dark theme</button>
<button data-commandLinker-command="filebrowser:go-to-path" data-commandlinker-args='{"path": "/workspace/llm-playground/"}' href="#">llm-playground</button>
<button data-commandLinker-command="filebrowser:go-to-path" data-commandlinker-args='{"path": "/workspace/axolotl/"}' href="#">axolotl</button>
<button data-commandLinker-command="filebrowser:go-to-path" data-commandlinker-args='{"path": "/workspace/llm-playground/notebooks/axolotl/runpod"}' href="#">Runpod notebooks</button>
<button data-commandLinker-command="filebrowser:go-to-path" data-commandlinker-args='{"path": "/workspace/axolotl/examples"}' href="#">axolotl configs</button>
<button data-commandLinker-command="filebrowser:go-to-path" data-commandlinker-args='{"path": "/workspace/llm-playground/storage"}' href="#">Storage</button>
<button data-commandLinker-command="filebrowser:go-to-path" data-commandlinker-args='{"path": "/content/axolotl-trained"}' href="#">axolotl-trained</button>
<button data-commandLinker-command="docmanager:open" data-commandlinker-args='{"path": "/workspace/axolotl/examples/falcon/config-40b-qlora.yml"}' href="#">Edit qlora config</button>
<button data-commandLinker-command="docmanager:open" data-commandlinker-args='{"path": "/workspace/axolotl/ds_config.json"}' href="#">Edit ds config</button>

## Prepare

### Set HF Cache

In [ ]:
# %env HF_DATASETS_CACHE

In [ ]:
#%env TRANSFORMERS_CACHE

In [ ]:
!rm -rf /root/.cache

In [ ]:
!mkdir -p /content/cache

In [ ]:
!ln -s /content/cache /root/.cache

### HF Login

In [ ]:
# For axolotl push_dataset_to_hub
import os
from huggingface_hub import notebook_login, login
# Colab:
# notebook_login()
# RunPod:
login(os.environ.get("HUGGINGFACE_TOKEN"), add_to_git_credential=True)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


### Update axolotl

In [ ]:
%cd /workspace/

/workspace


In [ ]:
!git clone https://github.com/OpenAccess-AI-Collective/axolotl axolotl-update

Cloning into 'axolotl-update'...
remote: Enumerating objects: 3145, done.
remote: Counting objects: 100% (1338/1338), done.
remote: Compressing objects: 100% (336/336), done.
remote: Total 3145 (delta 1084), reused 1129 (delta 952), pack-reused 1807
Receiving objects: 100% (3145/3145), 1.43 MiB | 2.24 MiB/s, done.
Resolving deltas: 100% (1973/1973), done.


In [ ]:
!cp -r axolotl-update/* axolotl

In [ ]:
%cd /workspace/axolotl

/workspace/axolotl


In [ ]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   scripts/runpod-entrypoint.sh

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	examples/falcon/ft.yml
	examples/falcon/lora.yml
	examples/falcon/qlora.yml
	examples/huggyllama/
	examples/openllama/

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
!ds_report

Setting ds_accelerator to cuda (auto detect)
--------------------------------------------------
DeepSpeed C++/CUDA extension op report
--------------------------------------------------
NOTE: Ops not installed will be just-in-time (JIT) compiled at
      runtime if needed. Op compatibility means that your system
      meet the required dependencies to JIT install the op.
--------------------------------------------------
JIT compiled ops requires ninja
ninja .................. [OKAY]
--------------------------------------------------
op name ................ installed .. compatible
--------------------------------------------------
async_io ............... [YES] ...... [OKAY]
cpu_adagrad ............ [YES] ...... [OKAY]
cpu_adam ............... [YES] ...... [OKAY]
fused_adam ............. [YES] ...... [OKAY]
fused_lamb ............. [YES] ...... [OKAY]
quantizer .............. [YES] ...... [OKAY]
random_ltd ............. [YES] ...... [OKAY]
 [WARNING]  sparse_attn requires a torch vers

In [ ]:
!pip install -e .

Obtaining file:///workspace/axolotl
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-install-awf5nfb1/transformers_914ce9d55534468ea1a220819448f7a6
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-install-awf5nfb1/transformers_914ce9d55534468ea1a220819448f7a6
  Resolved https://github.com/huggingface/transformers.git to commit 3723329d014a7b144863e597ea4fe6de5e6a8279
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Attempting uninstall: axolotl
    Found existing installation: axolotl 0.1
    Uninstalling axolotl-0.1:
      Successfully uninstalled axolotl-0.1
  Running setup.py develop for axolotl


In [ ]:
!pip list|grep torch

torch                    2.0.1
torchaudio               2.0.1+cu118
torchvision              0.15.1+cu118


### Init Storage

In [ ]:
!python /workspace/llm-playground/helper/storage.py utensil/axolotl-trained /content/ -m

Cloning https://huggingface.co/utensil/axolotl-trained into local empty directory.


In [ ]:
!ls /content/axolotl-trained

### Reinstall PyTorch with CUDA 11.8 (optional)

In [ ]:
!pip3 install -U torch --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 2.0 MB/s eta 0:00:00:00:0100:02
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1
    Uninstalling torch-2.0.1:
      Successfully uninstalled torch-2.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.1+cu118 requires torch==2.0.0, but you have torch 2.0.1+cu118 which is incompatible.


In [ ]:
!pip list|grep torch

torch                    2.0.1+cu118
torchaudio               2.0.1+cu118
torchvision              0.15.2


### Reinstall deepspeed (optional)

In [ ]:
!ds_report

Setting ds_accelerator to cuda (auto detect)
--------------------------------------------------
DeepSpeed C++/CUDA extension op report
--------------------------------------------------
NOTE: Ops not installed will be just-in-time (JIT) compiled at
      runtime if needed. Op compatibility means that your system
      meet the required dependencies to JIT install the op.
--------------------------------------------------
JIT compiled ops requires ninja
ninja .................. [OKAY]
--------------------------------------------------
op name ................ installed .. compatible
--------------------------------------------------
async_io ............... [YES] ...... [OKAY]
cpu_adagrad ............ [YES] ...... [OKAY]
cpu_adam ............... [YES] ...... [OKAY]
fused_adam ............. [YES] ...... [OKAY]
fused_lamb ............. [YES] ...... [OKAY]
quantizer .............. [YES] ...... [OKAY]
random_ltd ............. [YES] ...... [OKAY]
 [WARNING]  sparse_attn requires a torch vers

In [ ]:
!yes|pip uninstall deepspeed

yes: standard output: Broken pipe


In [ ]:
!TORCH_CUDA_ARCH_LIST="3.5;5.0;6.0;6.1;7.0;7.5;8.0;8.6+PTX" DS_BUILD_OPS=1 DS_BUILD_SPARSE_ATTN=0 pip install deepspeed --global-option="build_ext" --global-option="-j8" # --global-option="bdist_wheel"

DEPRECATION: --build-option and --global-option are deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use --config-settings. Discussion can be found at https://github.com/pypa/pip/issues/11859
  Using cached deepspeed-0.9.4.tar.gz (808 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [2491 lines of output]
      [2023-06-14 14:06:00,089] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
      [2023-06-14 14:06:01,651] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
      DS_BUILD_OPS=1
       [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.0
       [WARNING]  using untested triton version (2.0.0), only 1.0.0 is known to be compatible
       [WARNING]  Filtered compute capabilities ['3.5', '5.0']
      Install Ops={

In [ ]:
!pip install deepspeed

  Using cached deepspeed-0.9.4.tar.gz (808 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for deepspeed: filename=deepspeed-0.9.4-py3-none-any.whl size=843999 sha256=e8adebab399e3f0854f8064e3e5ef4f7fc1b08e3508321a48bf102c3a51ece79
  Stored in directory: /content/cache/pip/wheels/6a/60/33/8c1e5497fbf93e00c73a46e06fbbc12145131bc48820e86a40
Successfully built deepspeed


In [ ]:
!ds_report

[2023-06-14 14:09:57,707] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
--------------------------------------------------
DeepSpeed C++/CUDA extension op report
--------------------------------------------------
NOTE: Ops not installed will be just-in-time (JIT) compiled at
      runtime if needed. Op compatibility means that your system
      meet the required dependencies to JIT install the op.
--------------------------------------------------
JIT compiled ops requires ninja
ninja .................. [OKAY]
--------------------------------------------------
op name ................ installed .. compatible
--------------------------------------------------
async_io ............... [NO] ....... [OKAY]
cpu_adagrad ............ [NO] ....... [OKAY]
cpu_adam ............... [NO] ....... [OKAY]
fused_adam ............. [NO] ....... [OKAY]
fused_lamb ............. [NO] ....... [OKAY]
quantizer .............. [NO] ....... [OKAY]
random_ltd ....

### Init Configs

In [ ]:
%cd /workspace/axolotl

/workspace/axolotl


In [ ]:
# Try no config
# !accelerate config default

Setting ds_accelerator to cuda (auto detect)
accelerate configuration saved at /root/.cache/huggingface/accelerate/default_config.yaml


In [ ]:
%%writefile ds_config.json
{
    "zero_optimization": {
        "stage": 3,
        "offload_optimizer": {
            "device": "cpu",
            "pin_memory": true
        },
        "offload_param": {
            "device": "cpu",
            "pin_memory": true
        },
        "overlap_comm": true,
        "contiguous_gradients": true,
        "sub_group_size": 0,
        "reduce_bucket_size": "auto",
        "stage3_prefetch_bucket_size": "auto",
        "stage3_param_persistence_threshold": "auto",
        "stage3_max_live_parameters": 0,
        "stage3_max_reuse_distance": 0,
        "stage3_gather_16bit_weights_on_model_save": true
    },
   "bf16": {
        "enabled": "auto"
    },
    "fp16": {
        "enabled": "auto",
        "auto_cast": false,
        "loss_scale": 0,
        "initial_scale_power": 32,
        "loss_scale_window": 1000,
        "hysteresis": 2,
        "min_loss_scale": 1
    },
    "optimizer": {
        "type": "AdamW",
        "params": {
          "lr": "auto",
          "betas": "auto",
          "eps": "auto",
          "weight_decay": "auto"
        }
    },
    "scheduler": {
      "type": "WarmupDecayLR",
      "params": {
        "total_num_steps": "auto",
        "warmup_min_lr": "auto",
        "warmup_max_lr": "auto",
        "warmup_num_steps": "auto"
       }
    },
    "gradient_accumulation_steps": "auto",
    "train_batch_size": "auto",
    "train_micro_batch_size_per_gpu": "auto",
    "wall_clock_breakdown": false
}

Overwriting ds_config.json


In [ ]:
%%writefile examples/falcon/config-40b-qlora.yml
# 1b: tiiuae/falcon-rw-1b
# 7b: tiiuae/falcon-7b
# 40b: tiiuae/falcon-40b
base_model: tiiuae/falcon-40b
base_model_config: tiiuae/falcon-40b
# required by falcon custom model code: https://huggingface.co/tiiuae/falcon-7b/tree/main
trust_remote_code: true
model_type: AutoModelForCausalLM
tokenizer_type: AutoTokenizer
load_in_8bit: false
# enable 4bit for QLoRA
load_in_4bit: true
gptq: false
strict: false

push_dataset_to_hub: utensil
hf_use_auth_token: true

datasets:
  - path: QingyiSi/Alpaca-CoT
    data_files:
      - Chain-of-Thought/formatted_cot_data/gsm8k_train.json
    type: "alpaca:chat"

dataset_prepared_path: last_run_prepared
val_set_size: 0.01
# enable QLoRA
adapter: qlora
lora_model_dir:
sequence_len: 2048
max_packed_sequence_len:

# hyperparameters from QLoRA paper Appendix B.2
# "We find hyperparameters to be largely robust across datasets"
lora_r: 64
lora_alpha: 16
# 0.1 for models up to 13B
# 0.05 for 33B and 65B models
lora_dropout: 0.05
# add LoRA modules on all linear layers of the base model
lora_target_modules:
lora_target_linear: true
lora_fan_in_fan_out:

wandb_project: falcon-qlora
wandb_watch:
wandb_run_id:
wandb_log_model:
output_dir: /content/axolotl-trained/falcon-qlora-40b-gsm8k/

# QLoRA paper Table 9
# - 16 for 7b & 13b
# - 32 for 33b, 64 for 64b
# Max size tested on A6000
# - 7b: 40
# - 40b: 4
# decrease if OOM, increase for max VRAM utilization
micro_batch_size: 1
gradient_accumulation_steps: 1
num_epochs: 3
# Optimizer for QLoRA
# optimizer: paged_adamw_32bit
torchdistx_path:
# lr_scheduler: cosine
# QLoRA paper Table 9
# - 2e-4 for 7b & 13b
# - 1e-4 for 33b & 64b
learning_rate: 0.0002
train_on_inputs: false
group_by_length: false
bf16: true
fp16: false
tf32: true
gradient_checkpointing: true
# stop training after this many evaluation losses have increased in a row
# https://huggingface.co/transformers/v4.2.2/_modules/transformers/trainer_callback.html#EarlyStoppingCallback
early_stopping_patience: 3
resume_from_checkpoint:
auto_resume_from_checkpoints: true
local_rank:
logging_steps: 1
xformers_attention: true
flash_attention:
gptq_groupsize:
gptq_model_v1:
warmup_steps: 10
eval_steps: 5
save_steps: 10
debug:
deepspeed:
weight_decay: 0.01
fsdp:
fsdp_config:
special_tokens:
  pad_token: "<|endoftext|>"
  bos_token: ">>ABSTRACT<<"
  eos_token: "<|endoftext|>"

Overwriting examples/falcon/config-40b-qlora.yml


In [ ]:
%env ACCELERATE_USE_DEEPSPEED=true

env: ACCELERATE_USE_DEEPSPEED=true


In [ ]:
%%writefile scripts/ft.py
import os
from pathlib import Path
import fire
import logging
import finetune
from axolotl.utils.trainer import setup_trainer as setup_trainer_orig

logging.basicConfig(level=os.getenv("LOG_LEVEL", "INFO"))

def train_ex(
    config: Path = Path("configs/"),
    prepare_ds_only: bool = False,
    **kwargs,
):
  logging.info('train_ex before')
  finetune.train(config, prepare_ds_only, **kwargs)
  logging.info('train_ex after')

def setup_trainer_ex(cfg, train_dataset, eval_dataset, model, tokenizer):
  logging.info('setup_trainer_ex before')
  logging.info(f'cfg.some_config = {cfg.some_config}')
  trainer = setup_trainer_orig(cfg, train_dataset, eval_dataset, model, tokenizer)
  logging.info('setup_trainer_ex after')
  return trainer

finetune.setup_trainer = setup_trainer_ex

if __name__ == "__main__":
    fire.Fire(train_ex)

Writing scripts/ft.py


# Training

## #1

In [ ]:
%cd /workspace/axolotl

/workspace/axolotl


In [ ]:
!cat examples/falcon/config-40b-qlora.yml

In [ ]:
#%%writefile examples/falcon/config-40b-qlora.yml

In [ ]:
!accelerate launch scripts/finetune.py examples/falcon/config-40b-qlora.yml --deepspeed ds_config.json

Setting ds_accelerator to cuda (auto detect)
The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `2`
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
Setting ds_accelerator to cuda (auto detect)
Setting ds_accelerator to cuda (auto detect)

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues

===================================BUG REPORT===================================
Welc

## #2


In [ ]:
!cat examples/falcon/config-40b-qlora.yml

In [ ]:
!ds_report

Setting ds_accelerator to cuda (auto detect)
--------------------------------------------------
DeepSpeed C++/CUDA extension op report
--------------------------------------------------
NOTE: Ops not installed will be just-in-time (JIT) compiled at
      runtime if needed. Op compatibility means that your system
      meet the required dependencies to JIT install the op.
--------------------------------------------------
JIT compiled ops requires ninja
ninja .................. [OKAY]
--------------------------------------------------
op name ................ installed .. compatible
--------------------------------------------------
async_io ............... [YES] ...... [OKAY]
cpu_adagrad ............ [YES] ...... [OKAY]
cpu_adam ............... [YES] ...... [OKAY]
fused_adam ............. [YES] ...... [OKAY]
fused_lamb ............. [YES] ...... [OKAY]
quantizer .............. [YES] ...... [OKAY]
random_ltd ............. [YES] ...... [OKAY]
 [WARNING]  sparse_attn requires a torch vers

In [ ]:
#%%writefile examples/falcon/config-40b-qlora.yml

In [ ]:
!accelerate launch scripts/finetune.py examples/falcon/config-40b-qlora.yml --deepspeed ds_config.json

Setting ds_accelerator to cuda (auto detect)
The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `2`
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
Setting ds_accelerator to cuda (auto detect)
Setting ds_accelerator to cuda (auto detect)

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /root/miniconda3/envs/py3.9/lib/python3.9/site-packages/bitsandbytes-0.39.0-py3.9.

## #3 After reinstall deepspeed

In [ ]:
!cat examples/falcon/config-40b-qlora.yml

In [ ]:
#%%writefile examples/falcon/config-40b-qlora.yml

In [ ]:
!accelerate launch scripts/finetune.py examples/falcon/config-40b-qlora.yml --deepspeed ds_config.json

[2023-06-14 14:11:07,857] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `2`
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
[2023-06-14 14:11:11,854] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2023-06-14 14:11:11,880] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run



## #4

In [ ]:
%env ACCELERATE_USE_DEEPSPEED=false

env: ACCELERATE_USE_DEEPSPEED=false


In [ ]:
!cat examples/falcon/config-40b-qlora.yml

In [ ]:
#%%writefile examples/falcon/config-40b-qlora.yml

In [ ]:
!accelerate launch scripts/finetune.py examples/falcon/config-40b-qlora.yml

[2023-06-14 14:23:53,438] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `2`
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
[2023-06-14 14:23:56,823] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2023-06-14 14:23:56,905] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run



## #5 mbs 8

In [ ]:
%env HF_HUB_DISABLE_PROGRESS_BARS=1

env: HF_HUB_DISABLE_PROGRESS_BARS=1


In [ ]:
!cat examples/falcon/config-40b-qlora.yml

# 1b: tiiuae/falcon-rw-1b
# 7b: tiiuae/falcon-7b
# 40b: tiiuae/falcon-40b
base_model: tiiuae/falcon-40b
base_model_config: tiiuae/falcon-40b
# required by falcon custom model code: https://huggingface.co/tiiuae/falcon-7b/tree/main
trust_remote_code: true
model_type: AutoModelForCausalLM
tokenizer_type: AutoTokenizer
load_in_8bit: false
# enable 4bit for QLoRA
load_in_4bit: true
gptq: false
strict: false

push_dataset_to_hub: utensil
hf_use_auth_token: true

datasets:
  - path: QingyiSi/Alpaca-CoT
    data_files:
      - Chain-of-Thought/formatted_cot_data/gsm8k_train.json
    type: "alpaca:chat"

dataset_prepared_path: last_run_prepared
val_set_size: 0.01
# enable QLoRA
adapter: qlora
lora_model_dir:
sequence_len: 2048
max_packed_sequence_len:

# hyperparameters from QLoRA paper Appendix B.2
# "We find hyperparameters to be largely robust across datasets"
lora_r: 64
lora_alpha: 16
# 0.1 for models up to 13B
# 0.05 for 33B and 65B models
lora_dropout: 0.05
# add LoRA modules on all line

In [ ]:
!accelerate launch scripts/finetune.py examples/falcon/config-40b-qlora.yml

[2023-06-14 14:39:50,595] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `2`
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
[2023-06-14 14:39:54,179] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2023-06-14 14:39:54,193] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run



## #6

In [ ]:
!cat examples/falcon/config-40b-qlora.yml

In [ ]:
!accelerate launch scripts/finetune.py examples/falcon/config-40b-qlora.yml

[2023-06-14 14:48:55,999] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `2`
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
[2023-06-14 14:48:59,452] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2023-06-14 14:48:59,460] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run



## #7

In [ ]:
!cat examples/falcon/config-40b-qlora.yml

In [ ]:
!accelerate launch scripts/finetune.py examples/falcon/config-40b-qlora.yml --deepspeed ds_config.json

# Upload

### Upload checkpoints to HF

In [ ]:
%cd /content/axolotl-trained/falcon-qlora-40b-gsm8k/

In [ ]:
!ls -lhta |grep checkpoint-

In [ ]:
!ls -lhta |grep checkpoint- | awk 'NR > 1 {print $9}'

In [ ]:
# ls -lhta |grep checkpoint- | awk 'NR > 1 {print $9}' | xargs rm -rf

In [ ]:
!python /workspace/llm-playground/helper/storage.py utensil/axolotl-trained /content/ -u

## Below are ad hoc cells handling issues during training

current out dir:

```
/content/axolotl-trained/falcon-qlora-40b-gsm8k/
```

### Force release VRAM

In [ ]:
# First interupt the kernel, wait a few seconds then run this to kill finetune to release VRAM
!ps aux|grep python|grep finetune|awk '{print $2}'|xargs kill

### Clean the finetuned model and all checkpoints

In [ ]:
# Only run this to start over
!rm -rf /content/axolotl-trained/falcon-qlora-40b-gsm8k/

### Zip the prepared dataset

In [ ]:
!apt install zip
!zip -r last_run_prepared.zip -xi last_run_prepared

### Monitoring GPU

In [ ]:
# Run this in a seperate terminal
!nvitop -m full

### Fix DISK FULL

In [ ]:
%cd /

/


In [ ]:
!du -d 2 -h|grep G

In [ ]:
!du -d 2 -h /root/.local

In [ ]:
!rm -rf /root/.local/share/Trash/

In [ ]:
!rm -rf /root/.local/share/wandb/

In [ ]:
!rm -rf /root/.cache/wandb/

### Check who is using GPU

In [ ]:
!apt install lsof

In [ ]:
!lsof /dev/nvidia*

### A new bash without tmux etc.

In [ ]:
!bash --norc --noprofile

### Clean up all checkpoints but last one

In [ ]:
cd /content/axolotl-trained/falcon-qlora-40b-gsm8k/ && ls -lhta |grep checkpoint- | awk 'NR > 1 {print $9}' | xargs rm -rf